In [5]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical


In [6]:
data= pd.read_csv('./Desktop/Miscellenous/Titanic/train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Preprocessing the Data

In [7]:
# drop out unrelated columns
cols = data.drop(['PassengerId','Name', 'Ticket', 'Cabin','Survived'], axis=1)
cols.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [8]:
#replacing male with 1, female with 0 in Sex coulumn
cols['Sex'] = cols['Sex'].map({'male': 1, 'female': 0 })
cols.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,S
1,1,0,38.0,1,0,71.2833,C
2,3,0,26.0,0,0,7.9250,S
3,1,0,35.0,1,0,53.1000,S
4,3,1,35.0,0,0,8.0500,S


In [9]:
#convert categorical variable into dummy variables
features=pd.get_dummies(cols)
features.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,3,1,22.0,1,0,7.2500,0,0,1
1,1,0,38.0,1,0,71.2833,1,0,0
2,3,0,26.0,0,0,7.9250,0,0,1
3,1,0,35.0,1,0,53.1000,0,0,1
4,3,1,35.0,0,0,8.0500,0,0,1


In [10]:
#Explore data and check for missing values
features.info()
features.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Pclass        891 non-null int64
Sex           891 non-null int64
Age           714 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Embarked_C    891 non-null uint8
Embarked_Q    891 non-null uint8
Embarked_S    891 non-null uint8
dtypes: float64(2), int64(4), uint8(3)
memory usage: 44.5 KB


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,0.647587,29.699118,0.523008,0.381594,32.204208,0.188552,0.086420,0.722783
std,0.836071,0.477990,14.526497,1.102743,0.806057,49.693429,0.391372,0.281141,0.447876
min,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,20.125000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000
50%,3.000000,1.000000,28.000000,0.000000,0.000000,14.454200,0.000000,0.000000,1.000000
75%,3.000000,1.000000,38.000000,1.000000,0.000000,31.000000,0.000000,0.000000,1.000000
max,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000


In [11]:
#Age data is missing, so replacing it with mean
predictors = features.fillna(np.mean(features['Age']))
predictors.info()
print('shape of predictors is {}'.format(predictors.shape))
predictors.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Pclass        891 non-null int64
Sex           891 non-null int64
Age           891 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Embarked_C    891 non-null uint8
Embarked_Q    891 non-null uint8
Embarked_S    891 non-null uint8
dtypes: float64(2), int64(4), uint8(3)
memory usage: 44.5 KB
shape of predictors is (891, 9)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,3,1,22.0,1,0,7.2500,0,0,1
1,1,0,38.0,1,0,71.2833,1,0,0
2,3,0,26.0,0,0,7.9250,0,0,1
3,1,0,35.0,1,0,53.1000,0,0,1
4,3,1,35.0,0,0,8.0500,0,0,1


In [12]:
# 
#feature scaling
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# predictors = sc.fit_transform(predictors)

In [13]:
#convert predictors in nparray for Keras
predictors = predictors.values
target = to_categorical(data['Survived'].values)
#target = data['Survived'].values

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)


#### Model in Keras

In [15]:
# for Dropout regularization
from keras.layers import Dropout

In [16]:
#building the model architecure, model, compile & fit
cols = predictors.shape[1]
model = Sequential()
model.add(Dense(100, activation='relu', input_shape=(cols,)))
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
623/623 [==============================] - 1s 872us/step - loss: 2.0807 - acc: 0.5634
Epoch 2/100
623/623 [==============================] - 0s 164us/step - loss: 1.5714 - acc: 0.5843
Epoch 3/100
623/623 [==============================] - 0s 199us/step - loss: 1.1696 - acc: 0.6083
Epoch 4/100
623/623 [==============================] - 0s 183us/step - loss: 0.8813 - acc: 0.6292
Epoch 5/100
623/623 [==============================] - 0s 182us/step - loss: 0.8878 - acc: 0.6276
Epoch 6/100
623/623 [==============================] - 0s 198us/step - loss: 0.7576 - acc: 0.6661
Epoch 7/100
623/623 [==============================] - 0s 158us/step - loss: 0.7295 - acc: 0.6774
Epoch 8/100
623/623 [==============================] - 0s 203us/step - loss: 0.7063 - acc: 0.6661
Epoch 9/100
623/623 [==============================] - 0s 154us/step - loss: 0.6829 - acc: 0.6565
Epoch 10/100
623/623 [==============================] - 0s 202us/step - loss: 0.6574 - acc: 0.6726
Epoch 11/100
623/62

623/623 [==============================] - 0s 193us/step - loss: 0.4621 - acc: 0.8186
Epoch 84/100
623/623 [==============================] - 0s 174us/step - loss: 0.4763 - acc: 0.8138
Epoch 85/100
623/623 [==============================] - 0s 177us/step - loss: 0.4503 - acc: 0.8347
Epoch 86/100
623/623 [==============================] - 0s 164us/step - loss: 0.4768 - acc: 0.8138
Epoch 87/100
623/623 [==============================] - 0s 172us/step - loss: 0.4628 - acc: 0.8138
Epoch 88/100
623/623 [==============================] - 0s 162us/step - loss: 0.4421 - acc: 0.8266
Epoch 89/100
623/623 [==============================] - 0s 161us/step - loss: 0.4416 - acc: 0.8283
Epoch 90/100
623/623 [==============================] - 0s 200us/step - loss: 0.4560 - acc: 0.8250
Epoch 91/100
623/623 [==============================] - 0s 172us/step - loss: 0.4455 - acc: 0.8347
Epoch 92/100
623/623 [==============================] - 0s 230us/step - loss: 0.4439 - acc: 0.8250
Epoch 93/100
623/623 [=

In [17]:
y_pred = model.predict(X_test)
y_pred = (y_pred[:,1] >0.5)
y_test = (y_test[:,1])

In [18]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy = (cm[0,0] + cm[1,1])/(cm[0,0] + cm[1,1] + cm[0,1] + [cm[1,0]])
print('accuracy is {}%'.format(accuracy*100))

[[148   9]
 [ 49  62]]
accuracy is [ 78.35820896]%


#### K-fold Cross Validation with Dropout Regularization 

In [19]:
#import Keras wrapper fro sklearn and keras working
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [20]:
#defining the classifier function
def build_classifier():
    cols = predictors.shape[1]
    model = Sequential()
    model.add(Dense(100, activation='relu', input_shape=(cols,)))
    model.add(Dropout(0.3))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics = ['accuracy'])
    return model

In [25]:
classifier = KerasClassifier(build_fn=build_classifier, batch_size=10, epochs=100)
accuracies = cross_val_score(estimator=classifier, X = X_train, y=y_train, cv=10)

Epoch 1/100
560/560 [==============================] - 1s 1ms/step - loss: 2.0654 - acc: 0.5821
Epoch 2/100
560/560 [==============================] - 0s 179us/step - loss: 1.7530 - acc: 0.5821
Epoch 3/100
560/560 [==============================] - 0s 213us/step - loss: 1.5292 - acc: 0.5661
Epoch 4/100
560/560 [==============================] - 0s 172us/step - loss: 1.0832 - acc: 0.6464
Epoch 5/100
560/560 [==============================] - 0s 178us/step - loss: 1.0845 - acc: 0.6214
Epoch 6/100
560/560 [==============================] - 0s 219us/step - loss: 0.9233 - acc: 0.6518
Epoch 7/100
560/560 [==============================] - 0s 204us/step - loss: 0.7985 - acc: 0.6286
Epoch 8/100
560/560 [==============================] - 0s 170us/step - loss: 0.8454 - acc: 0.6768
Epoch 9/100
560/560 [==============================] - 0s 187us/step - loss: 0.7517 - acc: 0.6732
Epoch 10/100
560/560 [==============================] - 0s 167us/step - loss: 0.7426 - acc: 0.6589
Epoch 11/100
560/560 

560/560 [==============================] - 0s 211us/step - loss: 0.4560 - acc: 0.8232
Epoch 84/100
560/560 [==============================] - 0s 174us/step - loss: 0.4462 - acc: 0.8339
Epoch 85/100
560/560 [==============================] - 0s 211us/step - loss: 0.4600 - acc: 0.8107
Epoch 86/100
560/560 [==============================] - 0s 173us/step - loss: 0.4788 - acc: 0.8089
Epoch 87/100
560/560 [==============================] - 0s 213us/step - loss: 0.4498 - acc: 0.8107
Epoch 88/100
560/560 [==============================] - 0s 238us/step - loss: 0.4462 - acc: 0.8357
Epoch 89/100
560/560 [==============================] - 0s 219us/step - loss: 0.4455 - acc: 0.8304
Epoch 90/100
560/560 [==============================] - 0s 213us/step - loss: 0.4558 - acc: 0.8250
Epoch 91/100
560/560 [==============================] - 0s 215us/step - loss: 0.4728 - acc: 0.8054
Epoch 92/100
560/560 [==============================] - 0s 183us/step - loss: 0.4572 - acc: 0.8286
Epoch 93/100
560/560 [=

560/560 [==============================] - 0s 224us/step - loss: 0.4355 - acc: 0.8321
Epoch 66/100
560/560 [==============================] - 0s 191us/step - loss: 0.4645 - acc: 0.8161
Epoch 67/100
560/560 [==============================] - 0s 241us/step - loss: 0.4302 - acc: 0.8143
Epoch 68/100
560/560 [==============================] - 0s 227us/step - loss: 0.4350 - acc: 0.8161
Epoch 69/100
560/560 [==============================] - 0s 222us/step - loss: 0.4527 - acc: 0.8357
Epoch 70/100
560/560 [==============================] - 0s 200us/step - loss: 0.4206 - acc: 0.8321
Epoch 71/100
560/560 [==============================] - 0s 221us/step - loss: 0.4537 - acc: 0.8214
Epoch 72/100
560/560 [==============================] - 0s 220us/step - loss: 0.4444 - acc: 0.8286
Epoch 73/100
560/560 [==============================] - 0s 183us/step - loss: 0.4342 - acc: 0.8321
Epoch 74/100
560/560 [==============================] - 0s 188us/step - loss: 0.4280 - acc: 0.8375
Epoch 75/100
560/560 [=

560/560 [==============================] - 0s 211us/step - loss: 0.5211 - acc: 0.7696
Epoch 48/100
560/560 [==============================] - 0s 167us/step - loss: 0.5290 - acc: 0.7554
Epoch 49/100
560/560 [==============================] - 0s 139us/step - loss: 0.5120 - acc: 0.7857
Epoch 50/100
560/560 [==============================] - 0s 167us/step - loss: 0.5163 - acc: 0.7732
Epoch 51/100
560/560 [==============================] - 0s 140us/step - loss: 0.5047 - acc: 0.7893
Epoch 52/100
560/560 [==============================] - 0s 167us/step - loss: 0.5211 - acc: 0.7696
Epoch 53/100
560/560 [==============================] - 0s 167us/step - loss: 0.5021 - acc: 0.7946
Epoch 54/100
560/560 [==============================] - 0s 214us/step - loss: 0.5380 - acc: 0.7732
Epoch 55/100
560/560 [==============================] - 0s 211us/step - loss: 0.5164 - acc: 0.7750
Epoch 56/100
560/560 [==============================] - 0s 208us/step - loss: 0.4892 - acc: 0.7786
Epoch 57/100
560/560 [=

561/561 [==============================] - 0s 177us/step - loss: 0.5937 - acc: 0.7433
Epoch 30/100
561/561 [==============================] - 0s 204us/step - loss: 0.6323 - acc: 0.7219
Epoch 31/100
561/561 [==============================] - 0s 191us/step - loss: 0.5725 - acc: 0.7255
Epoch 32/100
561/561 [==============================] - 0s 182us/step - loss: 0.5339 - acc: 0.7469
Epoch 33/100
561/561 [==============================] - 0s 173us/step - loss: 0.6000 - acc: 0.7469
Epoch 34/100
561/561 [==============================] - 0s 172us/step - loss: 0.5320 - acc: 0.7807
Epoch 35/100
561/561 [==============================] - 0s 181us/step - loss: 0.5878 - acc: 0.7594
Epoch 36/100
561/561 [==============================] - 0s 181us/step - loss: 0.5396 - acc: 0.7629
Epoch 37/100
561/561 [==============================] - 0s 172us/step - loss: 0.5724 - acc: 0.7451
Epoch 38/100
561/561 [==============================] - 0s 173us/step - loss: 0.4966 - acc: 0.7861
Epoch 39/100
561/561 [=

...

561/561 [==============================] - 0s 171us/step - loss: 0.5074 - acc: 0.8004
Epoch 58/100
561/561 [==============================] - 0s 139us/step - loss: 0.4860 - acc: 0.8004
Epoch 59/100
561/561 [==============================] - 0s 167us/step - loss: 0.4749 - acc: 0.8182
Epoch 60/100
561/561 [==============================] - 0s 150us/step - loss: 0.5121 - acc: 0.8253
Epoch 61/100
561/561 [==============================] - 0s 208us/step - loss: 0.4831 - acc: 0.8111
Epoch 62/100
561/561 [==============================] - 0s 152us/step - loss: 0.4732 - acc: 0.8057
Epoch 63/100
561/561 [==============================] - 0s 167us/step - loss: 0.4898 - acc: 0.8039
Epoch 64/100
561/561 [==============================] - 0s 167us/step - loss: 0.4779 - acc: 0.8146
Epoch 65/100
561/561 [==============================] - 0s 160us/step - loss: 0.4654 - acc: 0.8128
Epoch 66/100
561/561 [==============================] - 0s 174us/step - loss: 0.4985 - acc: 0.8057
Epoch 67/100
561/561 [=

561/561 [==============================] - 0s 166us/step - loss: 0.5275 - acc: 0.7594
Epoch 40/100
561/561 [==============================] - 0s 171us/step - loss: 0.5387 - acc: 0.7790
Epoch 41/100
561/561 [==============================] - 0s 128us/step - loss: 0.5290 - acc: 0.7754
Epoch 42/100
561/561 [==============================] - 0s 139us/step - loss: 0.5404 - acc: 0.7861
Epoch 43/100
561/561 [==============================] - 0s 167us/step - loss: 0.5276 - acc: 0.7825
Epoch 44/100
561/561 [==============================] - 0s 167us/step - loss: 0.5641 - acc: 0.7807
Epoch 45/100
561/561 [==============================] - 0s 142us/step - loss: 0.5063 - acc: 0.7897
Epoch 46/100
561/561 [==============================] - 0s 162us/step - loss: 0.5227 - acc: 0.8039
Epoch 47/100
561/561 [==============================] - 0s 209us/step - loss: 0.5291 - acc: 0.7914
Epoch 48/100
561/561 [==============================] - 0s 200us/step - loss: 0.5025 - acc: 0.7897
Epoch 49/100
561/561 [=

561/561 [==============================] - 0s 183us/step - loss: 0.6200 - acc: 0.6881
Epoch 22/100
561/561 [==============================] - 0s 163us/step - loss: 0.6367 - acc: 0.6881
Epoch 23/100
561/561 [==============================] - 0s 191us/step - loss: 0.6366 - acc: 0.6952
Epoch 24/100
561/561 [==============================] - 0s 169us/step - loss: 0.6305 - acc: 0.6988
Epoch 25/100
561/561 [==============================] - 0s 189us/step - loss: 0.5881 - acc: 0.7041
Epoch 26/100
561/561 [==============================] - 0s 204us/step - loss: 0.6017 - acc: 0.7112
Epoch 27/100
561/561 [==============================] - 0s 144us/step - loss: 0.6051 - acc: 0.7326
Epoch 28/100
561/561 [==============================] - 0s 150us/step - loss: 0.6017 - acc: 0.7005
Epoch 29/100
561/561 [==============================] - 0s 167us/step - loss: 0.6075 - acc: 0.7077
Epoch 30/100
561/561 [==============================] - 0s 203us/step - loss: 0.5837 - acc: 0.7148
Epoch 31/100
561/561 [=

561/561 [==============================] - 0s 175us/step - loss: 1.1102 - acc: 0.6025
Epoch 4/100
561/561 [==============================] - 0s 209us/step - loss: 0.9346 - acc: 0.6132
Epoch 5/100
561/561 [==============================] - 0s 198us/step - loss: 0.8226 - acc: 0.6613
Epoch 6/100
561/561 [==============================] - 0s 207us/step - loss: 0.7863 - acc: 0.6399
Epoch 7/100
561/561 [==============================] - 0s 206us/step - loss: 0.7756 - acc: 0.6364
Epoch 8/100
561/561 [==============================] - 0s 200us/step - loss: 0.7822 - acc: 0.6435
Epoch 9/100
561/561 [==============================] - 0s 197us/step - loss: 0.6741 - acc: 0.6774
Epoch 10/100
561/561 [==============================] - 0s 175us/step - loss: 0.7161 - acc: 0.6738
Epoch 11/100
561/561 [==============================] - 0s 310us/step - loss: 0.6764 - acc: 0.6631
Epoch 12/100
561/561 [==============================] - 0s 172us/step - loss: 0.7022 - acc: 0.6667
Epoch 13/100
561/561 [=======

Epoch 86/100
561/561 [==============================] - 0s 193us/step - loss: 0.4734 - acc: 0.8111
Epoch 87/100
561/561 [==============================] - 0s 218us/step - loss: 0.4661 - acc: 0.8200
Epoch 88/100
561/561 [==============================] - 0s 243us/step - loss: 0.4812 - acc: 0.8021
Epoch 89/100
561/561 [==============================] - 0s 188us/step - loss: 0.4899 - acc: 0.8039
Epoch 90/100
561/561 [==============================] - 0s 175us/step - loss: 0.4588 - acc: 0.8039
Epoch 91/100
561/561 [==============================] - 0s 195us/step - loss: 0.4664 - acc: 0.7968
Epoch 92/100
561/561 [==============================] - 0s 167us/step - loss: 0.4920 - acc: 0.8182
Epoch 93/100
561/561 [==============================] - 0s 247us/step - loss: 0.4453 - acc: 0.8146
Epoch 94/100
561/561 [==============================] - 0s 215us/step - loss: 0.5007 - acc: 0.8200
Epoch 95/100
561/561 [==============================] - 0s 184us/step - loss: 0.5021 - acc: 0.8093
Epoch 96/1

In [26]:
print('accuracies are {}'.format(accuracies))
mean = accuracies.mean()
variance = accuracies.std()
print('mean = {}'.format(mean))
print('variance = {}'.format(variance))#loading the test data
test = pd.read_csv('./Desktop/Miscellenous/Titanic/test.csv')
test.head()

accuracies are [ 0.7936508   0.76190477  0.95238095  0.87096773  0.77419355  0.74193548
  0.77419355  0.77419355  0.82258064  0.88709677]
mean = 0.8153097781748022
variance = 0.06392900616649881


In [28]:
#loading the test data
test = pd.read_csv('./Desktop/Miscellenous/Titanic/test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


#### Preprocessing the Test Data

In [29]:
cols_test = test.drop(['PassengerId','Name', 'Ticket', 'Cabin'], axis=1)
cols_test['Sex'] = cols_test['Sex'].map({'male': 1, 'female': 0 })
features_test=pd.get_dummies(cols_test)
features_test.head()
predictors_test = features_test.fillna(np.mean(features_test['Age']))
predictors_test.info()
print('shape of predictors_test is {}'.format(predictors_test.shape))
predictors_test = predictors_test.values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
Pclass        418 non-null int64
Sex           418 non-null int64
Age           418 non-null float64
SibSp         418 non-null int64
Parch         418 non-null int64
Fare          418 non-null float64
Embarked_C    418 non-null uint8
Embarked_Q    418 non-null uint8
Embarked_S    418 non-null uint8
dtypes: float64(2), int64(4), uint8(3)
memory usage: 20.9 KB
shape of predictors_test is (418, 9)


In [30]:
#get predicted surival probability for test data
pred = model.predict(predictors_test)
pred_prob_survived = pred[:,1]
pred_prob_survived = (pred_prob_survived>0.5)

In [31]:
df = pd.DataFrame(pred_prob_survived)
df.to_csv("./Desktop/Miscellenous/Titanic/submit.csv")